In [1]:
from bs4 import BeautifulSoup
import requests
import time
import random
import pandas as pd

In [2]:
def GetGenreByTitle(artist,title):

    url = "https://www.google.com/search?q="
    headers = {'User-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134"}
    
    artist = artist.lower()
    artist = artist.replace(" ","+")
    title = title.lower()
    title = title.replace(" ","+")
    
    search = url+title+"+"+artist+"+genre"
    try:
        page = requests.get(search,headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser')
    
        mydivs = soup.find_all("div", {"class":"Z0LcW XcVN5d"})

        if len(mydivs) == 1:
            genre = [str(mydivs)[27:-7]]
        else:
            mydivs = soup.find_all("div", {"class":"rlc__slider-page"})
            genre = []
            for element in range(len(mydivs)):
                genre_title = mydivs[element].find("div", {"class":"title"})
                genre_title = str(genre_title)[19:-6]
                genre.append(genre_title)
    except:
        genre = "Search for Genre failed"
    time.sleep(1)
    return genre

In [3]:
data = pd.DataFrame(columns = ["Artist","Song_title","Song_text","Song_genre"])

# Website

In [4]:
website = "https://www.songtexte.de/musiker.html?seite="

In [5]:
weblist = []
for x in range(1,51):
    weblist.append(website+str(x))

In [6]:
# Dict bestehend aus {Seitenzahl:{Künstler:[link],...}...}

In [7]:
songdict = {}
counter = 1
for url in weblist:
    songdict.update({counter:{}})
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    mydivs = soup.find_all("div", {"class": "cover"})
    for c in range(len(mydivs)):
        tag = mydivs[c].a.attrs
        link = tag["href"]
        artist = tag["title"][:-34]
        songdict[counter].update({artist:link})
    counter += 1
    time.sleep(random.random()*1+1)

In [8]:
# Dict bestehend aus {{Künstler:{Liedname:[Liedurl]},...}...}

In [9]:
artistdict = {}
counter = 0
fail = {}

for element in range(1,len(songdict.keys())+1): #Für jede Seite
    for artist in songdict[element].keys(): #Für jeden Künstler
        counter += 1
        songs = {}
        try:
            url = songdict[element][artist]
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')
            mydivs = soup.find_all("div", {"class": "album-content col-sm-9"})[0]
            mydivs = mydivs.find("ul")
            tags = mydivs.find_all("a")
            #print(len(tags))
            for c in range(len(tags)): #Für jeden Song des Künstlers
                song = tags[c]["title"][len(artist)+3:-9]
                link = tags[c]["href"]
                songs.update({song:[link]})
                artistdict.update({artist:songs})
            #print(counter, artist, "erfolgreich")
        except:
            fail.update({artist:songdict[element][artist]})
            print(counter, artist, "fehlgeschlagen Link:", songdict[element][artist])

        counter += 1
        time.sleep(random.random()*1+2)

625 Various Artists fehlgeschlagen Link: https://www.songtexte.de/musiker/various-artists-69323.html
687 Lindsey Stirling fehlgeschlagen Link: https://www.songtexte.de/musiker/lindsey-stirling-2234532.html
789 Journey fehlgeschlagen Link: https://www.songtexte.de/musiker/journey-1988812.html
1013 周杰倫 fehlgeschlagen Link: https://www.songtexte.de/musiker/535341.html
1395 No Doubt fehlgeschlagen Link: https://www.songtexte.de/musiker/no-doubt-568745.html
1555 Brand New fehlgeschlagen Link: https://www.songtexte.de/musiker/brand-new-162841.html
1681 Macklemore fehlgeschlagen Link: https://www.songtexte.de/musiker/macklemore-2289198.html
1863 Fedez fehlgeschlagen Link: https://www.songtexte.de/musiker/fedez-857656.html
2197 Vetusta Morla fehlgeschlagen Link: https://www.songtexte.de/musiker/vetusta-morla-2307448.html
2567 Project Pat fehlgeschlagen Link: https://www.songtexte.de/musiker/project-pat-2611536.html
2819 Fat Joe fehlgeschlagen Link: https://www.songtexte.de/musiker/fat-joe-7743

In [10]:
# Dict bestehend aus {{Künstler:{Liedname:[Liedurl,Lyrics],...}...}

In [11]:
counter = 0
fail2 = {}
for artist in artistdict.keys():
    for song in artistdict[artist]:
        try:
            url = artistdict[artist][song][0]
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')
            mydivs = soup.find_all("div", {"class": "album-content col-sm-9 col-xs-12"})
            tag = mydivs[0].find_all("p")[1]
            song_text = str(tag)[23:-22].replace("<br>","").replace("<br/>","")
            #artistdict[artist][song].append(song_text)
            genre = GetGenreByTitle(artist,song)
            data[counter] = [artist,song,song_text,genre]             
            counter +=1
            time.sleep(random.random()*1+0.5)
        except:
            fail2.update({artist:{song:url}})
            print("Fehler bei", artist, song, url)
        if counter%500 == 0:
            print(counter,"Lieder heruntergeladen")
        

Fehler bei Drake  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fdrake-614110.html
Fehler bei Drake bo //www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fdrake-614110.html
Fehler bei Drake sA whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fdrake-614110.html
500 Lieder heruntergeladen
Fehler bei Kygo  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fkygo-2088817.html
Fehler bei Kygo ebo //www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fkygo-2088817.html
Fehler bei Kygo tsA whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fkygo-2088817.html
Fehler bei Sam Martin  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fsam-martin-2288925.html
Fehler bei Conrad  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fconrad-223752.html
Fehler bei Conrad o //www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fconrad-223752.html
Fehler b

Fehler bei Classical Study Music  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fclassical-study-music-265190.html
10000 Lieder heruntergeladen
Fehler bei Lost Frequencies  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Flost-frequencies-768596.html
Fehler bei Hayley Williams  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fhayley-williams-968695.html
Fehler bei Sebastian Ingrosso  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fsebastian-ingrosso-1068714.html
10500 Lieder heruntergeladen
Fehler bei Remo  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fremo-2719890.html
Fehler bei Remo ebo //www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fremo-2719890.html
Fehler bei Remo tsA whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fremo-2719890.html
Fehler bei RAVVEL  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fravvel-2759759.html
Fehler bei RAVVEL 

Fehler bei Travi$ Scott  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Ftravi-scott-2858794.html
Fehler bei Astrid Lindgren  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fastrid-lindgren-10.html
Fehler bei Sam-E  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fsam-e-11.html
Fehler bei Sam-E bo //www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fsam-e-11.html
Fehler bei Sam-E sA whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fsam-e-11.html
Fehler bei Prince & The Revolution  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fprince-the-revolution-11352.html
Fehler bei Ayah Marar  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fayah-marar-11375.html
16000 Lieder heruntergeladen
Fehler bei Amber Coffman  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Famber-coffman-16938.html
Fehler bei Anna Kendrick  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%

20000 Lieder heruntergeladen
Fehler bei Maffio  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fmaffio-2299070.html
Fehler bei Maffio o //www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fmaffio-2299070.html
Fehler bei Maffio A whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fmaffio-2299070.html
Fehler bei MNEK  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fmnek-2305835.html
Fehler bei MNEK ebo //www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fmnek-2305835.html
Fehler bei MNEK tsA whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fmnek-2305835.html
Fehler bei OIAM  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Foiam-2489026.html
Fehler bei OIAM ebo //www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Foiam-2489026.html
Fehler bei OIAM tsA whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Foiam-2489026.html
Fehler bei O

Fehler bei Anders Nilsen  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fanders-nilsen-5821.html
Fehler bei Amba Shepherd  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Famba-shepherd-16903.html
Fehler bei AKA AKA  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Faka-aka-16944.html
Fehler bei Agatha Lee Monn  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fagatha-lee-monn-22465.html
Fehler bei Arty  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Farty-22466.html
Fehler bei Arty ebo //www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Farty-22466.html
Fehler bei Arty tsA whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Farty-22466.html
Fehler bei Sultan & Ned Shepard  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fsultan-ned-shepard-22477.html
Fehler bei Chimamanda Ngozi Adichie  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fchimamanda

Fehler bei Chris Malinchak  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fchris-malinchak-240685.html
26500 Lieder heruntergeladen
Fehler bei Carnage  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fcarnage-513068.html
Fehler bei Callejeros  whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fcallejeros-548228.html
Fehler bei CD9  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fcd9-566340.html
Fehler bei CD9 cebo //www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fcd9-566340.html
Fehler bei CD9 atsA whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fcd9-566340.html
Fehler bei DMA  //twitter.com/share?url=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fdma-568464.html
Fehler bei DMA cebo //www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fdma-568464.html
Fehler bei DMA atsA whatsapp://send?text=https%3A%2F%2Fwww.songtexte.de%2Fmusiker%2Fdma-568464.html
Fehler bei Dalmat

In [12]:
#artistdict["Ed Sheeran"]['Thinking Out Loud'][0]

In [13]:
# counter = 0
# for artist in artistdict.keys():
#     for song in artistdict[artist]:
#         if len(artistdict[artist][song]) == 2:
#             genre = GetGenreByTitle(artist,song)
#             tracks_df[counter] = [artist,song,artistdict[artist][song][1],genre]
#             counter +=1

In [51]:
data.head(20)

,Artist,Song_title,Song_text,Song_genre
0,Ed Sheeran,Thinking Out Loud,When your looks don't work like they used to b...,"[Blue-Eyed Soul, Pop]"
1,Ed Sheeran,Sing,[Verse 1]\nIt's late in the evening\nGlass on ...,[]
2,Ed Sheeran,I'm A Mess,"[Verse 1]\nOh, I'm a mess right now, inside ou...",[Pop]
3,Ed Sheeran,One,[Verse 1]\nTell me that you'll turn down the m...,[Pop]
4,Ed Sheeran,Afire Love,Things were all good yesterday\r\nAnd then the...,[Pop]
5,Ed Sheeran,Nina,[Produced by Jake Gosling]\n\n[Verse 1]\nI met...,[Pop]
6,Ed Sheeran,Runaway,[Verse 1]\nI've known it for a long time\nDadd...,[Pop]
7,Ed Sheeran,The A Team,"White lips, pale face, breathing in snowflakes...",[Pop]
8,Ed Sheeran,Take It Back,I'm not a rapper; I'm a singer with a flow\nI'...,[Pop]
9,Ed Sheeran,Even My Dad Does Sometimes,It's alright to cry\r\nEven my dad does someti...,[Pop]


In [55]:
data.to_pickle("niklas_data_001_050.pkl")